In [ ]:
from IPython.display import Markdown, clear_output
import ipywidgets as widgets
from protomol.rd import mol
from protomol.util import render, update_db
from protomol.util.backend import Append_SQL, Query_SQL, Styles, write_orca


def on_query_smiles_btn(b):
    clear_output()
    display(header_box)
    if smiles_cbo.value not in smiles_cbo.options:
        display(write_smiles_box)
        render.from_smiles(smiles_cbo.value, add_labels=True)
    else:
        calcs = Query_SQL.calculations(smiles_cbo.value)
        if len(calcs) == 0:
            sh_cmd = write_orca(smiles_cbo.value, 2)
            display(Markdown(sh_cmd))
        else:
            display(calcs_box)


def on_write_smiles_btn(b):
    Append_SQL.smiles(smiles_cbo.value, multip_int.value)
    clear_output()
    display(
        f"Added {smiles_cbo.value} with multiplicity {multip_int.value} to data.db. Please restart."
    )


def on_refresh_db_btn(b):
    update_db.refresh()


def on_visualize_xyz_btn(b):
    clear_output()
    display(header_box, calcs_box)
    rows = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value)
    if len(rows) == 1:
        alpha_indices, beta_indices = mol.beta_cleavage_indices(smiles_cbo.value)
        alpha_cbo.options = alpha_indices
        beta_cbo.options = beta_indices
        display(new_calc_box)
        render.from_xyz_block(Query_SQL.xyzs(rows[0][0]), add_labels=True)
    else:
        display("No matching XYZ in database.")


def on_submit_opt_btn(b):
    rows = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value)
    if len(rows) == 1:
        sh_cmd = write_orca(smiles_cbo.value, new_methods_cbo.value, rows[0][0])
        display(Markdown(sh_cmd))
    else:
        display("No matching XYZ in database.")


# Header box
smiles_cbo = widgets.Combobox(
    options=Query_SQL.smiles(), layout=Styles.scale_width("50%")
)
query_smiles_btn = widgets.Button(
    description="Submit Query", layout=Styles.scale_width("20%")
)
query_smiles_btn.on_click(on_query_smiles_btn)
refresh_db_btn = widgets.Button(
    description="Refresh Database", layout=Styles.scale_width("20%")
)
refresh_db_btn.on_click(on_refresh_db_btn)
header_box = widgets.HBox([smiles_cbo, query_smiles_btn, refresh_db_btn])

# New SMILES box
multip_int = widgets.IntText(
    description="Multiplicity: ", layout=Styles.scale_width("20%")
)
write_smiles_btn = widgets.Button(
    description="Write SMILES to database", layout=Styles.scale_width("20%")
)
write_smiles_btn.on_click(on_write_smiles_btn)
write_smiles_box = widgets.HBox([multip_int, write_smiles_btn])

# Methods Box
methods_cbo = widgets.Dropdown(
    options=Query_SQL.methods(), layout=Styles.scale_width("50%")
)
visualize_xyz_btn = widgets.Button(
    description="Visualize XYZ", layout=Styles.scale_width("20%")
)
visualize_xyz_btn.on_click(on_visualize_xyz_btn)
visualize_traj_btn = widgets.Button(
    description="Visualize Traj", layout=Styles.scale_width("20%")
)
calcs_box = widgets.HBox([methods_cbo, visualize_xyz_btn, visualize_traj_btn])

# New Calculation Box
new_methods_cbo = widgets.Dropdown(
    options=Query_SQL.methods(),
    description="New Method",
    layout=Styles.scale_width("40%"),
)
alpha_cbo = widgets.Dropdown(
    options=[], description="α", layout=Styles.scale_width("10%")
)
beta_cbo = widgets.Dropdown(
    options=[], description="β", layout=Styles.scale_width("10%")
)
submit_opt_btn = widgets.Button(
    description="Submit Optimization", layout=Styles.scale_width("20%")
)
submit_opt_btn.on_click(on_submit_opt_btn)
submit_path_btn = widgets.Button(
    description="Submit Path", layout=Styles.scale_width("20%")
)
new_calc_box = widgets.HBox(
    [new_methods_cbo, alpha_cbo, beta_cbo, submit_opt_btn, submit_path_btn]
)

display(header_box)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
functional = "CCSD(T)-F12/RI"
basis = "cc-pVDZ-F12 cc-pVDZ-F12-CABS cc-pVTZ/c"
method = "CALC"
inp_template = (
    "%PAL NPROCS [num_cpus] END\n"
    "%maxcore [mem_per_cpu]\n"
    "! CCSD(T)-F12/RI cc-pVDZ-F12 cc-pVDZ-F12-CABS cc-pVTZ/c\n"
    "* xyzfile 0 [multiplicity] init.xyz\n"
)
submit_template = (
    "#!/bin/bash\n"
    "#SBATCH --partition=batch\n"
    "#SBATCH --gres=lscratch:[lscratch_size]\n"
    "#SBATCH --job-name=[SMILES]_CALC\n"
    "#SBATCH --nodes=1\n"
    "#SBATCH --ntasks=[num_cpus]\n"
    "#SBATCH --ntasks-per-node=[num_cpus]\n"
    "#SBATCH --cpus-per-task=1\n"
    "#SBATCH --time=[time_requested]\n"
    "#SBATCH --mem-per-cpu=[mem_per_cpu]G\n\n"
    "cd ${SLURM_SUBMIT_DIR}\n"
    "mkdir -p /lscratch/${USER}/${SLURM_JOB_ID}\n"
    'cp -r "${SLURM_SUBMIT_DIR}/." /lscratch/${USER}/${SLURM_JOB_ID}\n'
    "cd /lscratch/${USER}/${SLURM_JOB_ID}\n\n"
    "module load ORCA/6.1\n"
    "$(which orca) CALC.inp > CALC.log\n"
    "cp -r /lscratch/${USER}/${SLURM_JOB_ID} ${SLURM_SUBMIT_DIR}\n"
    "rm -rf /lscratch/${USER}/${SLURM_JOB_ID}\n"
)

Append_SQL._method(functional, basis, method, inp_template, submit_template)